In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q /content/drive/MyDrive/IN-Abs-small.zip -d /content/

In [3]:
!pip install --quiet transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [4]:
test_judgement_dir = '/content/IN-Abs-small/test-data/judgement'
test_summary_dir = '/content/IN-Abs-small/test-data/summary'

In [5]:
import os

In [6]:
output_path = '/content/drive/MyDrive/IN-Abs-small-results'
os.makedirs(output_path, exist_ok=True)
# Save your results there

In [10]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from evaluate import load

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Define device first
print(f"Using device: {device}")

model_name = 't5-base'  # You can switch to t5-small or t5-large as needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
rouge = load('rouge')

# 4. Function to chunk text into max 512 tokens
def chunk_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True, clean_up_tokenization_spaces=True) for chunk in chunks]

# 5. Summarize chunks and aggregate
def summarize_chunks(chunks):
    summaries = []
    for chunk in chunks:
        input_ids = tokenizer('summarize: ' + chunk, return_tensors='pt', truncation=True, max_length=512).input_ids.to(device)
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return ' '.join(summaries)

# 6. Paths for test-data
test_judgement_dir = '/content/IN-Abs-small/test-data/judgement'
test_summary_dir = '/content/IN-Abs-small/test-data/summary'

# 7. Run summarization and ROUGE evaluation
all_rouge_scores = []

for filename in os.listdir(test_judgement_dir):
    if not filename.endswith('.txt'):
        continue

    # Read judgement file and chunk
    with open(os.path.join(test_judgement_dir, filename), 'r', encoding='utf-8') as f:
        judgement_text = f.read()

    chunks = chunk_text(judgement_text, max_tokens=512)
    generated_summary = summarize_chunks(chunks)

    # Read reference summary
    with open(os.path.join(test_summary_dir, filename), 'r', encoding='utf-8') as f:
        reference_summary = f.read()

    # Compute ROUGE
    scores = rouge.compute(predictions=[generated_summary], references=[reference_summary])
    all_rouge_scores.append((filename, scores))

    print(f"File: {filename}\nROUGE Scores: {scores}\n")


Using device: cuda
File: 5937.txt
ROUGE Scores: {'rouge1': np.float64(0.6469002695417789), 'rouge2': np.float64(0.5311653116531165), 'rougeL': np.float64(0.5390835579514824), 'rougeLsum': np.float64(0.6361185983827493)}

File: 2440.txt
ROUGE Scores: {'rouge1': np.float64(0.525027808676307), 'rouge2': np.float64(0.2594654788418709), 'rougeL': np.float64(0.22691879866518355), 'rougeLsum': np.float64(0.4360400444938821)}

File: 7109.txt
ROUGE Scores: {'rouge1': np.float64(0.3930510314875136), 'rouge2': np.float64(0.21545157780195864), 'rougeL': np.float64(0.24972855591748105), 'rougeLsum': np.float64(0.34310532030401736)}

File: 4963.txt
ROUGE Scores: {'rouge1': np.float64(0.36923076923076925), 'rouge2': np.float64(0.2596401028277635), 'rougeL': np.float64(0.28717948717948716), 'rougeLsum': np.float64(0.3435897435897436)}

File: 2649.txt
ROUGE Scores: {'rouge1': np.float64(0.5354523227383863), 'rouge2': np.float64(0.23529411764705885), 'rougeL': np.float64(0.25916870415647913), 'rougeLsum

In [18]:
# Get keys from the dict inside the first tuple
keys = all_rouge_scores[0][1].keys()

avg_rouge = {}
for key in keys:
    avg_rouge[key] = (
        sum(scores[1][key] for scores in all_rouge_scores)
        / len(all_rouge_scores)
    )

print("Average ROUGE scores across all test files:")
for key, val in avg_rouge.items():
    print(f"{key}: {val:.4f}")


Average ROUGE scores across all test files:
rouge1: 0.4715
rouge2: 0.2368
rougeL: 0.2559
rougeLsum: 0.4042


In [8]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b4db1e01dd68ce5a490f6b762e3643cfde58f6e750a4996e712a2730e3b8d0ee
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


testing few summaries

In [19]:
for i, filename in enumerate(os.listdir(test_judgement_dir)):
    if not filename.endswith('.txt'):
        continue

    # Read judgement file and chunk
    with open(os.path.join(test_judgement_dir, filename), 'r', encoding='utf-8') as f:
        judgement_text = f.read()

    chunks = chunk_text(judgement_text, max_tokens=512)
    generated_summary = summarize_chunks(chunks)

    # Read reference summary
    with open(os.path.join(test_summary_dir, filename), 'r', encoding='utf-8') as f:
        reference_summary = f.read()

    # Compute ROUGE
    scores = rouge.compute(predictions=[generated_summary], references=[reference_summary])
    all_rouge_scores.append((filename, scores))

    # Print ROUGE and summaries for first 3 files only
    if i < 3:
        print(f"\n=== File: {filename} ===")
        print(f"Generated Summary:\n{generated_summary}\n")
        print(f"Reference Summary:\n{reference_summary}\n")
        print(f"ROUGE Scores: {scores}\n")



=== File: 5937.txt ===
Generated Summary:
the value of the processed fabric would be the value of the grey cloth in the hands of the processor . the factory gate here means the "deemed" factory gate as if the processed fabric was sold by the processor . if the trader gave a declaration to the processor as to what would be the price at which he would be selling the processed goods in the market . the price at which he is selling the goods must be the value of the grey cloth or fabric plus the value of the job work done plus the manufacturing profit and the manufacturing expenses . it is necessary to include the processor's expenses, costs and charges plus profit, but it is not necessary to include the trader's profits who gets the fabrics processed .

Reference Summary:
On a civil miscellaneous petition for clarification of this Court 's judgment dated 4th November, 1988, the Court.
HELD: The assessable value of the processed fabric would be the value of the grey cloth in the hands of 